import pandas as pd
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from settings import RAW_PATH
from sklearn import preprocessing
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelBinarizer
import numpy as np
import category_encoders as ce
from pandas import Series
from pandas import DataFrame
import ast
from typing import Dict, List
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import GridSearchCV
from settings import DATA_PATH
from datetime import datetime

In [73]:
#!pip install plotly

In [74]:
df = pd.read_csv(DATA_PATH /"df_text.csv", index_col= 0, parse_dates=['publish_date'])
df.sort_values('publish_date', inplace=True)
df['date'] = df['publish_date'].apply(lambda _date:_date.date())
df.head()

,title,publish_date,session,authors,ctr,category,tags,views,depth,full_reads_percent,...,max_ctr,new_title,coef,coef_third,text_words_amount,text_avg_word_len,text_word_len_10_ratio,text_word_len_8_ratio,text_word_len_5_ratio,date
document_id,,,,,,,,,,,,,,,,,,,,,
55c3984d9a79470f2c260465BKmAn_fHQruT38beqDIE_A,"['развод', 'миллион', 'российский', 'бизнесмен...",2017-08-07 13:00:27,BKmAn_fHQruT38beqDIE_A,['57c3fa919a7947e69071e942'],1.422,5433e5decbb20f277b20eca9,"['5409f144e063daa0f408ad26', '5409f3ade063daa0...",6238,1.067,54.489,...,0.0,"['российский', 'расходиться']",0.001041,1,620,7.116129,0.074194,0.296774,0.746774,2017-08-07
5a8eb3149a7947e3e34bba2dbgzlzJ2GTQacZgtv4WEb1A,"['мок', 'дать', 'мок', 'брать', 'атлет', 'лиша...",2018-02-22 14:28:30,bgzlzJ2GTQacZgtv4WEb1A,['5829aef49a7947876bda9414'],0.668,5433e5decbb20f277b20eca9,"['5409f144e063daa0f408ad26', '544fc342cbb20f41...",14558,1.150,38.295,...,0.0,"['дать', 'брать', 'лишаться', 'олимпийский']",0.002081,1,900,7.328889,0.107778,0.346667,0.742222,2018-02-22
605321259a7947663a3b6ab2vgBnAMPBRIOYrUrK0JeEiQ,"['гагаринский', 'одиссея', 'первый', 'космонав...",2021-04-12 11:01:26,vgBnAMPBRIOYrUrK0JeEiQ,['5f71b0719a794740b0ce312f'],0.657,5433e5decbb20f277b20eca9,"['5409f144e063daa0f408ad26', '54dc85719a794779...",10160,1.160,43.770,...,0.0,"['гагаринский', 'первый', 'путешествовать']",0.003122,1,806,6.955335,0.110422,0.254342,0.674938,2021-04-12
609d47119a794795c5f7617dh5z7vG_TQAu5Q7Nwp35LUw,"['человек', 'комсомольский', 'агрессивный', 'т...",2021-05-14 11:19:24,h5z7vG_TQAu5Q7Nwp35LUw,"['5f80446c9a794717f5424cdb','5f8435739a7947731...",5.675,5409f11ce063da9c8b588a12,"['599ecfca9a79472da7fab0dd', '5409f15de063daa0...",52682,1.088,28.467,...,0.0,"['комсомольский', 'агрессивный', 'известный']",0.016393,1,1116,7.619176,0.151434,0.347670,0.745520,2021-05-14
5afd8e7f9a794772862de199oXkM3UjQQRK7NZTlmVi63A,"['улюкаев', 'сердюкова', 'российский', 'чиновн...",2021-05-17 14:40:01,oXkM3UjQQRK7NZTlmVi63A,['596360489a7947894da539d3'],0.670,5409f11ce063da9c8b588a12,"['5409f144e063daa0f408ad26', '5409f146e063daa0...",4640,1.057,31.164,...,0.0,['российский'],0.000596,1,808,7.372525,0.167079,0.309406,0.719059,2021-05-17


In [75]:
df = df[df['views'] <= 1_000_000]
df = df[df['depth'] < 1.79]
df.loc[df['full_reads_percent'] > 100, 'full_reads_percent'] = np.nan
df['full_reads_percent'].fillna((df['full_reads_percent'].mean()), inplace=True)

df = df[df.category.isin(
    ['5409f11ce063da9c8b588a18', '5409f11ce063da9c8b588a12', '5433e5decbb20f277b20eca9', '540d5ecacbb20f2524fc050a',
     '540d5eafcbb20f2524fc0509', '5409f11ce063da9c8b588a13'])]

In [76]:
df['Time'] = np.arange(len(df.index))
df.head()

,title,publish_date,session,authors,ctr,category,tags,views,depth,full_reads_percent,...,new_title,coef,coef_third,text_words_amount,text_avg_word_len,text_word_len_10_ratio,text_word_len_8_ratio,text_word_len_5_ratio,date,Time
document_id,,,,,,,,,,,,,,,,,,,,,
55c3984d9a79470f2c260465BKmAn_fHQruT38beqDIE_A,"['развод', 'миллион', 'российский', 'бизнесмен...",2017-08-07 13:00:27,BKmAn_fHQruT38beqDIE_A,['57c3fa919a7947e69071e942'],1.422,5433e5decbb20f277b20eca9,"['5409f144e063daa0f408ad26', '5409f3ade063daa0...",6238,1.067,54.489,...,"['российский', 'расходиться']",0.001041,1,620,7.116129,0.074194,0.296774,0.746774,2017-08-07,0
5a8eb3149a7947e3e34bba2dbgzlzJ2GTQacZgtv4WEb1A,"['мок', 'дать', 'мок', 'брать', 'атлет', 'лиша...",2018-02-22 14:28:30,bgzlzJ2GTQacZgtv4WEb1A,['5829aef49a7947876bda9414'],0.668,5433e5decbb20f277b20eca9,"['5409f144e063daa0f408ad26', '544fc342cbb20f41...",14558,1.150,38.295,...,"['дать', 'брать', 'лишаться', 'олимпийский']",0.002081,1,900,7.328889,0.107778,0.346667,0.742222,2018-02-22,1
605321259a7947663a3b6ab2vgBnAMPBRIOYrUrK0JeEiQ,"['гагаринский', 'одиссея', 'первый', 'космонав...",2021-04-12 11:01:26,vgBnAMPBRIOYrUrK0JeEiQ,['5f71b0719a794740b0ce312f'],0.657,5433e5decbb20f277b20eca9,"['5409f144e063daa0f408ad26', '54dc85719a794779...",10160,1.160,43.770,...,"['гагаринский', 'первый', 'путешествовать']",0.003122,1,806,6.955335,0.110422,0.254342,0.674938,2021-04-12,2
609d47119a794795c5f7617dh5z7vG_TQAu5Q7Nwp35LUw,"['человек', 'комсомольский', 'агрессивный', 'т...",2021-05-14 11:19:24,h5z7vG_TQAu5Q7Nwp35LUw,"['5f80446c9a794717f5424cdb','5f8435739a7947731...",5.675,5409f11ce063da9c8b588a12,"['599ecfca9a79472da7fab0dd', '5409f15de063daa0...",52682,1.088,28.467,...,"['комсомольский', 'агрессивный', 'известный']",0.016393,1,1116,7.619176,0.151434,0.347670,0.745520,2021-05-14,3
5afd8e7f9a794772862de199oXkM3UjQQRK7NZTlmVi63A,"['улюкаев', 'сердюкова', 'российский', 'чиновн...",2021-05-17 14:40:01,oXkM3UjQQRK7NZTlmVi63A,['596360489a7947894da539d3'],0.670,5409f11ce063da9c8b588a12,"['5409f144e063daa0f408ad26', '5409f146e063daa0...",4640,1.057,31.164,...,['российский'],0.000596,1,808,7.372525,0.167079,0.309406,0.719059,2021-05-17,4


In [77]:
df=df[df['views']<1_000_000]

In [78]:
cats={

'540d5ecacbb20f2524fc050a':'медиа и технологии',
    '5409f11ce063da9c8b588a12':'политика',
'5433e5decbb20f277b20eca9':'общество',
'540d5eafcbb20f2524fc0509':'бизнес',
    '5409f11ce063da9c8b588a18':'финансы',
'5409f11ce063da9c8b588a13':'экономика'}


In [79]:
import plotly.graph_objects as go

In [80]:
_df=df[df['date']>datetime.strptime('05-01-2022', '%d-%m-%Y').date()]

In [81]:
popular_cats=['Военная операция на Украине',
'Политика',
'Общество',
'Бизнес',
'Война санкций',
'Пандемия коронавируса',
'Экономика',
'Технологии и медиа',
'Финансы',
'Город',
'Высылки дипломатов',
'Конфликт в Донбассе',
'Протесты в Казахстане',
'Протесты в Белоруссии',
'Конфликт Армении и Азербайджана',
'Рост цен на газ',
'Выборы президента Франции',
'Дело Навального',
'Доходы власти',
# 'Дискуссионный клуб',
# 'Дело Порошенко и Медведчука',
# 'Отставки губернаторов',
# 'Протесты в Армении',
# 'Конфликт в Афганистане',
# 'Дело Абызова',
# 'ПМЭФ-2022',
# 'День выборов',
# 'Конфликт в Нагорном Карабахе'
             ]

def replace_sub_cat(row):
    sub_cat = row['sub_cat']
    if sub_cat not in popular_cats:
         row['sub_cat'] = 'rare_sub_cat'
    return row

In [82]:
_df= _df.apply(lambda row: replace_sub_cat(row), axis=1)

In [ ]:
_df= _df.apply(lambda row: replace_sub_cat(row), axis=1)

In [ ]:
cats = _df['sub_cat'].unique()

In [84]:
cats = _df['sub_cat'].unique()

In [85]:
fig = go.Figure([{
    'x': _df[_df['sub_cat'] == cat_name]['date'],
    'y': _df[_df['sub_cat'] == cat_name]['views'],
    'name': cat_name
}  for  cat_name in cats])
fig.show(renderer='iframe')

In [86]:
fig = go.Figure([{
    'x': _df[_df['sub_cat'] == cat_name]['date'],
    'y': _df[_df['sub_cat'] == cat_name]['depth'],
    'name': cat_name
}  for  cat_name in cats])
fig.show(renderer='iframe')

In [87]:
fig = go.Figure([{
    'x': _df[_df['sub_cat'] == cat_name]['date'],
    'y': _df[_df['sub_cat'] == cat_name]['full_reads_percent'],
    'name': cat_name
}  for  cat_name in cats])
fig.show(renderer='iframe')

In [40]:
fig = go.Figure([{
    'x': _df[_df['category'] == cat_id]['date'],
    'y': _df[_df['category'] == cat_id]['depth'],
    'name': cat_name
}  for cat_id, cat_name in cats.items()])
fig.show(renderer='iframe')

In [41]:
fig = go.Figure([{
    'x': _df[_df['category'] == cat_id]['date'],
    'y': _df[_df['category'] == cat_id]['full_reads_percent'],
    'name': cat_name
}  for cat_id, cat_name in cats.items()])
fig.show(renderer='iframe')

In [42]:
df['day'] = pd.to_datetime(df['publish_date']).dt.strftime("%d").astype(int)
df['month'] = pd.to_datetime(df['publish_date']).dt.strftime("%m").astype(int)
df['hour'] = pd.to_datetime(df['publish_date']).dt.strftime("%H").astype(int)

In [ ]:
rs = np.random.RandomState(0)
corr = df.corr()
corr=corr.replace(1.000000, -0.01)
corr.style.background_gradient(cmap='coolwarm')

In [33]:
rs = np.random.RandomState(0)
corr = df.corr()
corr=corr.replace(1.000000, -0.01)
corr.style.background_gradient(cmap='coolwarm')

,ctr,views,depth,full_reads_percent,Time,day,month,hour
ctr,-0.010000,0.383622,-0.119505,-0.304219,0.091412,0.068100,0.102844,0.111155
views,0.383622,-0.010000,0.270729,-0.077288,-0.249445,0.100927,-0.225456,-0.002586
depth,-0.119505,0.270729,-0.010000,0.243519,-0.700095,-0.273354,-0.657874,-0.020042
full_reads_percent,-0.304219,-0.077288,0.243519,-0.010000,-0.111066,-0.034383,-0.090243,0.000939
Time,0.091412,-0.249445,-0.700095,-0.111066,-0.010000,0.321541,0.936126,0.008850
day,0.068100,0.100927,-0.273354,-0.034383,0.321541,-0.010000,0.119220,0.019438
month,0.102844,-0.225456,-0.657874,-0.090243,0.936126,0.119220,-0.010000,-0.001110
hour,0.111155,-0.002586,-0.020042,0.000939,0.008850,0.019438,-0.001110,-0.010000
